# Ein KI-Chatbot in 5 Minuten

Dieses Notebook führt dich durch folgende Schritte:

- Installation der benötigten Pakete
- Benutzen des Sitemap Loaders um die deepshore.de nach Wissensbeiträgen zu durchkämmen
- Die Dokumente an ein Embeddings Modell schicken und in Vectoren verwandeln und im Index speichern 
- Eine Frage an das Embeddings Modell schicken und die neu gewonnenen Vectoren zusammen mit den indexierten Vectoren verorten und eine Antwort bekommen

Du benötigst:

- python 3.10
- Jupyter Notebook Server
- Einen OpenAI API Token


## Installation

In [ ]:
%pip install llama_index=="0.6.38"
%pip install llama_hub=="0.0.5"
%pip install mercury=="2.3.4"

import mercury as mr
 
# set Application parameters
app = mr.App(title="",
        description="",
        show_code=False,
        show_prompt=False,
        continuous_update=True,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

## Open AI Konfiguration

### warum openai?

[llama_index](https://github.com/jerryjliu/llama_index) und anad llm-app-frameworks benutzen meist als Standard die openai high-level api für embeddings (vektorisierung von daten).

![](images/chatbot_graph-white-bg-v3.png)

In [ ]:
import os
import logging
import sys
import getpass

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Dokumente laden

... indem man den Sitemap Loaders benutzt, um die Webseite deepshore.de nach Wissensbeiträgen zu durchkämmen

In [ ]:
from llama_hub.web.sitemap.base import SitemapReader

import nest_asyncio
nest_asyncio.apply()

loader = SitemapReader(html_to_text=True)
documents = loader.load_data(sitemap_url='https://deepshore.de/sitemap.xml', filter='https://deepshore.de/knowledge')

print(len(documents))

## Die Dokumente an ein Embeddings Modell schicken

...und in Vektoren verwandeln und im Index speichern 

### warum vektor index?
ein vector index speichert/indexiert vektordaten für eine schnellere abfrage oder filterung (similarity search) von Daten, die sonst jedes mal gegen das ursprüngliche Modell gesendet werden müssen.
   
![](images/how-does-vector-store-work.png)

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()

## Eine Frage an das Modell schicken

... und eine Antwort im Kontext der Daten bekommen

In [ ]:
import textwrap

question = mr.Text(value="Was ist k6.io? Wofür benutzt man es?", label="Womit kann Deepshore helfen?", rows=1)

query_engine = index.as_query_engine()

response = query_engine.query(question.value)

messages = [question.value, "\n".join(textwrap.wrap(response.response,29))]
mr.Chat(messages)